In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, PowerTransformer
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectPercentile, SelectFromModel, mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score, GridSearchCV
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)

import joblib
from sklearn.metrics import accuracy_score

from pprint import pprint

In [2]:
train = pd.read_csv('./data/titanic_train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
train.Survived.value_counts(True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

In [4]:
# SibSp, Parch, Cabin edits
train['SibSp'] = train['SibSp'].apply(lambda x: 2 if x > 2 else x)
train['Parch'] = train['Parch'].apply(lambda x: 2 if x > 2 else x)
train['Cabin'] = train['Cabin'].apply(lambda x: 0 if pd.isna(x) else 1)

---
---
### Build pipeline
- cat_pipe for categorical variables:
    - Impute modal values where necessary
    - One-Hot encode
- num_pipe for numerical variables:
    - Impute mean values where necessary
    - Power transform to get more Gaussian-like distributions (important for logistic regression, good point to discuss)
    - Scale
- col_transformer:
    - cat_pipe
    - num_pipe
    - Drop extra columns we don't want to be used
- final pipe (`pipe`):
    - col_transformer
    - Logistic regression

#### Other notes:
- Imbalance in target classes maybe not so big that need to resort to imblearn (SMOTE or similar method)
- I tried adding class_weight='balanced' parameter but it made the accuracy scores worse
- I tried adding a feature_selection step to the final pipeline but both SelectPercentile and SelectFromModel made the accuracy score worse. I guess this is due to there being very few features in this dataset anyway.
    - useful to have as a trial to see how/where these methods can be added to the pipeline and know that they don't throw errors
- Also tried a random forest classifier as the final estimator, with the same preprocessing steps, and got worse accuracy with the test set

In [5]:
droppers = ['PassengerId','Name','Ticket']
cat_cols = ['Pclass','Sex','SibSp','Parch','Cabin','Embarked']
num_cols = ['Age','Fare']

cat_pipe = Pipeline(steps = [
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('oneHot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

num_pipe = Pipeline(steps = [
    ('impute', SimpleImputer(strategy='mean')),
    ('power', PowerTransformer(method='yeo-johnson')),
    ('scale', StandardScaler())
])

col_transformer = ColumnTransformer(transformers = [
    ('cat_pipe', cat_pipe, cat_cols),
    ('num_pipe', num_pipe, num_cols),
    ('drop', 'drop', droppers)
])

lr_pipe = Pipeline(steps = [
    ('process', col_transformer),
    #('selection', SelectPercentile(mutual_info_classif, percentile=50)),
    #('selection', SelectFromModel(RandomForestClassifier(random_state=42))),
    ('logisticregression', LogisticRegression())
])

---
---
### Fit pipeline and cross validate on test set

In [6]:
# Don't need to train_test_split as we have the test set in separate csv???
target = 'Survived'
X = train.drop(target, axis=1)
y = train[target]

In [7]:
cv_scores = cross_val_score(lr_pipe, X, y, cv=5, scoring='accuracy')

print(f'Cross Validation Accuracy: {(np.mean(cv_scores) *100):.2f}%')

Cross Validation Accuracy: 81.03%


---
---
### Tune hyperparamters with GridSearchCV

In [8]:
param_grids = [
    {
        'logisticregression__solver': ['saga'],
        'logisticregression__penalty': ['elasticnet'],
        'logisticregression__C': [0.1, 0.5, 1.0, 2.0],
        'logisticregression__l1_ratio': [0.1, 0.4, 0.6, 0.9],  # Only for elasticnet
        'logisticregression__max_iter': [100, 500, 1000, 10000]
    },
    {
        'logisticregression__solver': ['liblinear'],
        'logisticregression__penalty': ['l1', 'l2'],  # Only l1 and l2 supported
        'logisticregression__C': [0.1, 0.5, 1.0, 2.0],
        'logisticregression__max_iter': [100, 500, 1000, 10000]
    },
    {
        'logisticregression__solver': ['lbfgs'],
        'logisticregression__penalty': ['l2'],
        'logisticregression__C': [0.1, 0.5, 1.0, 2.0],
        'logisticregression__max_iter': [100, 500, 1000, 10000]
    }
]

gs = GridSearchCV(estimator=lr_pipe, param_grid=param_grids, cv=5, n_jobs=1, scoring='accuracy')
gs.fit(X, y)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('process',
                                        ColumnTransformer(transformers=[('cat_pipe',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('oneHot',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         ['Pclass',
                                                                          'Sex',
                                                                          'SibSp',
                                                                          'Parch',
                                                                          'Cabin',
                                                                          'Embarked']),
                                                                        ('num_pipe',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer()),
                                                                                         ('po...
                         {'logisticregression__C': [0.1, 0.5, 1.0, 2.0],
                          'logisticregression__max_iter': [100, 500, 1000,
                                                           10000],
                          'logisticregression__penalty': ['l1', 'l2'],
                          'logisticregression__solver': ['liblinear']},
                         {'logisticregression__C': [0.1, 0.5, 1.0, 2.0],
                          'logisticregression__max_iter': [100, 500, 1000,
                                                           10000],
                          'logisticregression__penalty': ['l2'],
                          'logisticregression__solver': ['lbfgs']}],
             scoring='accuracy')

### Best hyperparamsters and accuracy score from GridSearchCV

In [9]:
print('Best HyperParameters:')
pprint(gs.best_params_)
print()
print(f'Best Accuracy Score: {(gs.best_score_ *100):.2f}%')


Best HyperParameters:
{'logisticregression__C': 2.0,
 'logisticregression__l1_ratio': 0.1,
 'logisticregression__max_iter': 100,
 'logisticregression__penalty': 'elasticnet',
 'logisticregression__solver': 'saga'}

Best Accuracy Score: 81.59%


In [10]:
lr_clf = gs.best_estimator_

In [11]:
# Save fitted and tuned pipeline
joblib.dump(lr_clf, './saved_pipelines/titanic_lr_will.pkl')

['./saved_pipelines/titanic_lr_will.pkl']

---
---
### Evaluate against test set

In [12]:
test = pd.read_csv('./data/titanic_test.csv')

test['SibSp'] = test['SibSp'].apply(lambda x: 2 if x > 2 else x)
test['Parch'] = test['Parch'].apply(lambda x: 2 if x > 2 else x)
test['Cabin'] = test['Cabin'].apply(lambda x: 0 if pd.isna(x) else 1)

X_test = test.drop(target, axis=1)
y_test = test[target]

In [13]:
y_preds = lr_clf.predict(X_test)
test_acc = accuracy_score(y_test, y_preds)

print(f'Logistic Regression Test Accuracy Score: {(test_acc * 100):.2f}%')

Logistic Regression Test Accuracy Score: 75.60%
